## Okay, dann starten wir mit dem replizieren der Ergebnisse!
Als erstes wüssten wir gerne, ob die Werte der top 1-Accuracy und der Variational Distance mit denen auf Tabelle 13,17,14,18
Fangen wir an die Werte der Variational Distance zu verlgeichen mit denen aus Tabelle 14 und 18, hier zuerst mit TinyStories da der Datensatz kleiner ist.

In [48]:
#HIER ist der richtige Code, allerdings ist das dummy model echt schlecht und es gibt nie übereinstimmungen mit den regeln
import torch
import pandas as pd
import numpy as np
import sentencepiece as spm
import gcsfs
from transformers import AutoModelForCausalLM

# GPU aktivieren, falls verfügbar
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Google Cloud Storage verbinden
fs = gcsfs.GCSFileSystem('transformer-ngrams')

# Tokenizer laden
TOKENIZER_PATH = 'gs://transformer-ngrams/32768.model'
VOCAB_SIZE = 32768
BOS_TOKEN = 1
with fs.open(TOKENIZER_PATH) as f:
    tokenizer = spm.SentencePieceProcessor(model_proto=f.read())

# Transformer-Modell laden, unser trainiertes Modell
model_name = "dadosbon/gtp2wiki_tinymodel"
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

# Funktion für Modellvorhersagen (angepasst für Geschwindigkeit)
def get_model_predictions(input_tokens):
    input_tensor = torch.tensor([input_tokens]).to(device)
    with torch.no_grad():
        outputs = model(input_tensor)
        logits = outputs.logits[:, -1, :VOCAB_SIZE]
        probs = torch.nn.functional.softmax(logits, dim=-1)

                # Debugging-Ausgabe
        print("Logits (min, max):", logits.min().item(), logits.max().item())
        print("Erste 3 Logits:", logits[0, :3])
    return probs.cpu().numpy().flatten()

def convert_counter_to_probs_topk(arr, k=3):
    probs = np.zeros(VOCAB_SIZE)

    if arr is None or len(arr) == 0:
        return probs  # Falls leer, nur Nullen zurückgeben!

    arr = np.array(arr).flatten()

    # Falls die Länge ungerade ist, entfernen wir das letzte Element
    if len(arr) % 2 != 0:
        arr = arr[:-1]

    try:
        tokens = arr[::2].astype(int)  # Token-IDs
        counts = arr[1::2].astype(int)  # Häufigkeiten
    except Exception as e:
        print("Fehler beim Extrahieren von Tokens/Counts:", e)
        return probs  # Falls ein Fehler auftritt, bleibt es 0


    if len(tokens) == 0 or len(counts) == 0:
        return probs  # Falls etwas schiefgeht, alle Wahrscheinlichkeiten auf 0 setzen

    total_count = np.sum(counts)
    if total_count == 0:
        return probs  # Falls alle Counts 0 sind, bleibt es 0!

    token_probs = {int(token): count / total_count for token, count in zip(tokens, counts) if count > 0}
    topk_tokens = sorted(token_probs, key=token_probs.get, reverse=True)[:k]

    for token, prob in token_probs.items():
        if 0 <= token < VOCAB_SIZE:
            probs[token] = prob  # Wahrscheinlichkeiten setzen

    return probs  # Falls leer, nur Nullen zurückgeben!

    arr = np.array(arr).flatten()

    # Falls die Länge ungerade ist, entfernen wir das letzte Element
    if len(arr) % 2 != 0:
        arr = arr[:-1]

    tokens = arr[::2]  # Token-IDs
    counts = arr[1::2]  # Häufigkeiten

    if len(tokens) == 0 or len(counts) == 0:
        return probs  # Falls etwas schiefgeht, alle Wahrscheinlichkeiten auf 0 setzen

    total_count = np.sum(counts)
    if total_count == 0:
        return probs  # Falls alle Counts 0 sind, bleibt es 0!

    token_probs = {int(token): count / total_count for token, count in zip(tokens, counts) if count > 0}
    topk_tokens = sorted(token_probs, key=token_probs.get, reverse=True)[:k]

    for token in topk_tokens:
        if 0 <= token < VOCAB_SIZE:
            probs[token] = token_probs[token]  # Wahrscheinlichkeiten setzen

    return probs  # Falls leer, nur Nullen zurückgeben!

    arr = np.array(arr).flatten()

    # Falls die Länge ungerade ist, entfernen wir das letzte Element
    if len(arr) % 2 != 0:
        arr = arr[:-1]

    tokens = arr[::2]  # Token-IDs
    counts = arr[1::2]  # Häufigkeiten

    total_count = np.sum(counts)
    if total_count == 0:
        return probs  # Falls alle Counts 0 sind, bleibt es 0!

    token_probs = {int(token): count / total_count for token, count in zip(tokens, counts)}
    topk_tokens = sorted(token_probs, key=token_probs.get, reverse=True)[:k]

    for token in topk_tokens:
        if 0 <= token < VOCAB_SIZE:
            probs[token] = token_probs[token]  # Wahrscheinlichkeiten setzen

    return probs
    arr = np.array(arr).flatten()
    if arr.ndim != 1 or len(arr) % 2 != 0:
        return probs
    tokens = arr[::2]
    counts = arr[1::2]
    total_count = np.sum(counts)
    if total_count == 0:
        return probs
    token_probs = {int(token): count / total_count for token, count in zip(tokens, counts)}
    topk_tokens = sorted(token_probs, key=token_probs.get, reverse=True)[:k]
    for token in topk_tokens:
        if token < VOCAB_SIZE:
            probs[token] = token_probs[token]
    return probs

def dist(counter, model_probs):
    probs = convert_counter_to_probs_topk(counter)
    return 0.5 * np.sum(np.abs(probs - model_probs[:VOCAB_SIZE]))

# Regeln-Daten laden
print("Laden der Regeln-Daten für Tabelle 13...")
sample_rules_path = 'gs://transformer-ngrams/TinyStories/eval_data_rules/001.parquet'
with fs.open(sample_rules_path, 'rb') as f:
    df_rules = pd.read_parquet(f)
print("Regeln-Daten geladen:", df_rules.shape)

# Modellvorhersagen generieren
df_rules_sample = df_rules.sample(n=1000, random_state=42).copy()
model_probs = []
for token_list in df_rules_sample["token"]:
    probs = get_model_predictions([token_list])
    model_probs.append(probs.tolist())

df_rules_sample["model_probs"] = model_probs

df_rules_sample["distance"] = df_rules_sample.apply(
    lambda row: dist(row["next_token_counter"], row["model_probs"]), axis=1
)

df_rules_sample["rule_prediction"] = df_rules_sample["next_token_counter"].apply(convert_counter_to_probs_topk)
df_rules_sample["model_top_1"] = df_rules_sample["model_probs"].apply(np.argmax)
df_rules_sample["matches_rule"] = df_rules_sample.apply(
    lambda x: x["model_top_1"] in np.argsort(x["rule_prediction"])[-3:], axis=1
)

# Gruppierung nach Kontextlänge für Tabellen 13 & 14
df_rules_sample["context_length"] = df_rules_sample["context"].apply(
    lambda x: len([t for t in x.tolist() if t != 0]) if isinstance(x, np.ndarray) else 1
)
table_results = df_rules_sample.groupby("context_length").agg(
    top1_accuracy=("matches_rule", "mean"),
    avg_distance=("distance", "mean")
).reset_index()

table_results.rename(columns={"context_length": "Regeln / Kontextlänge", 
                               "top1_accuracy": "Top-1 Genauigkeit", 
                               "avg_distance": "Variationale Distanz"}, inplace=True)

# Speichern der Ergebnisse
table_results.to_csv("tiny_stories_results.csv", index=False)
print("✅ Ergebnisse gespeichert in 'tiny_stories_results.csv'") #-> hoffentlich hier mit neuem model bessere werte. sonst neu tokenisieren?

Using device: cuda


config.json:   0%|          | 0.00/752 [00:00<?, ?B/s]

C:\Users\larir\miniconda3\envs\projekt\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\larir\.cache\huggingface\hub\models--dadosbon--gtp2wiki_tinymodel. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/1.35G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

Laden der Regeln-Daten für Tabelle 13...
Regeln-Daten geladen: (28073, 11)
Logits (min, max): -9.431739807128906 6.694937229156494
Erste 3 Logits: tensor([-0.6478,  4.0172, -8.6157], device='cuda:0')
Logits (min, max): -9.536467552185059 7.079527378082275
Erste 3 Logits: tensor([-1.2776,  4.3381, -8.4441], device='cuda:0')
Logits (min, max): -9.778934478759766 6.71535587310791
Erste 3 Logits: tensor([-1.2305,  4.4007, -8.9970], device='cuda:0')
Logits (min, max): -9.387036323547363 6.989542007446289
Erste 3 Logits: tensor([-0.6856,  3.8047, -8.5453], device='cuda:0')
Logits (min, max): -9.536467552185059 7.079527378082275
Erste 3 Logits: tensor([-1.2776,  4.3381, -8.4441], device='cuda:0')
Logits (min, max): -9.594892501831055 6.891966342926025
Erste 3 Logits: tensor([-1.0305,  4.1471, -8.7263], device='cuda:0')
Logits (min, max): -9.43426513671875 7.010594844818115
Erste 3 Logits: tensor([-0.9592,  3.8428, -8.5228], device='cuda:0')
Logits (min, max): -9.488941192626953 6.700423717498

# Debugging 
Warum sind top1 und distance 0 und 1?

In [49]:
for idx, row in df_rules_sample.head(10).iterrows():
    print(f"Row {idx}: {convert_counter_to_probs_topk(row['next_token_counter'])}")



Row 10034: [0. 0. 0. ... 0. 0. 0.]
Row 22703: [0. 0. 0. ... 0. 0. 0.]
Row 14069: [0. 0. 0. ... 0. 0. 0.]
Row 11812: [0. 0. 0. ... 0. 0. 0.]
Row 667: [0. 0. 0. ... 0. 0. 0.]
Row 3711: [0. 0. 0. ... 0. 0. 0.]
Row 3553: [0. 0. 0. ... 0. 0. 0.]
Row 22215: [0. 0. 0. ... 0. 0. 0.]
Row 19861: [0. 0. 0. ... 0. 0. 0.]
Row 10957: [0. 0. 0. ... 0. 0. 0.]


In [50]:
print(df_rules_sample["rule_prediction"].head(10))


10034    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
22703    [0.0, 0.0, 0.0, 0.0, 0.008524240809802876, 0.0...
14069    [0.0, 0.0, 0.0, 0.0, 0.0014479979853941074, 0....
11812    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
667      [0.0, 0.0, 0.0, 0.0, 0.018710077046417584, 0.0...
3711     [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
3553     [0.0, 0.0, 0.0, 0.0, 9.21545739386865e-06, 0.0...
22215    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
19861    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
10957    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
Name: rule_prediction, dtype: object


In [51]:
print(df_rules_sample[["model_top_1"]].head(10))


       model_top_1
10034          280
22703          280
14069          280
11812          280
667            280
3711           280
3553           280
22215          280
19861          280
10957          280


In [52]:
print(df_rules_sample.apply(lambda x: np.argsort(x["rule_prediction"])[-3:], axis=1).head(10))
print(df_rules_sample[["model_top_1"]].head(10))


10034      [590, 4405, 32604]
22703      [2883, 13086, 305]
14069        [977, 356, 1947]
11812       [985, 32619, 459]
667         [32599, 305, 280]
3711     [32765, 8495, 12700]
3553      [32626, 498, 32600]
22215         [275, 280, 306]
19861         [620, 275, 496]
10957      [1264, 32619, 459]
dtype: object
       model_top_1
10034          280
22703          280
14069          280
11812          280
667            280
3711           280
3553           280
22215          280
19861          280
10957          280


In [53]:
# Debugging: Zeige die ersten 10 Zeilen mit Vergleich von model_top_1 und rule_prediction
for i, row in df_rules_sample.head(10).iterrows():
    top1 = row["model_top_1"]
    rule_probs = row["rule_prediction"]
    top3_rule_tokens = np.argsort(rule_probs)[-3:][::-1]  # Die Top-3 Token nach Regelwahrscheinlichkeit

    print(f"Row {i}: Model Top-1 = {top1}, Rule Top-3 = {top3_rule_tokens}")

    if top1 in top3_rule_tokens:
        print("Model Top-1 ist in den Rule Top-3!")
    else:
        print("xxxxModel Top-1 NICHT in den Rule Top-3!xxxx")




Row 10034: Model Top-1 = 280, Rule Top-3 = [32604  4405   590]
xxxxModel Top-1 NICHT in den Rule Top-3!xxxx
Row 22703: Model Top-1 = 280, Rule Top-3 = [  305 13086  2883]
xxxxModel Top-1 NICHT in den Rule Top-3!xxxx
Row 14069: Model Top-1 = 280, Rule Top-3 = [1947  356  977]
xxxxModel Top-1 NICHT in den Rule Top-3!xxxx
Row 11812: Model Top-1 = 280, Rule Top-3 = [  459 32619   985]
xxxxModel Top-1 NICHT in den Rule Top-3!xxxx
Row 667: Model Top-1 = 280, Rule Top-3 = [  280   305 32599]
Model Top-1 ist in den Rule Top-3!
Row 3711: Model Top-1 = 280, Rule Top-3 = [12700  8495 32765]
xxxxModel Top-1 NICHT in den Rule Top-3!xxxx
Row 3553: Model Top-1 = 280, Rule Top-3 = [32600   498 32626]
xxxxModel Top-1 NICHT in den Rule Top-3!xxxx
Row 22215: Model Top-1 = 280, Rule Top-3 = [306 280 275]
Model Top-1 ist in den Rule Top-3!
Row 19861: Model Top-1 = 280, Rule Top-3 = [496 275 620]
xxxxModel Top-1 NICHT in den Rule Top-3!xxxx
Row 10957: Model Top-1 = 280, Rule Top-3 = [  459 32619  1264]
xxxx

In [54]:
def get_model_predictions(input_tokens):
    input_tensor = torch.tensor([input_tokens]).to(device)
    with torch.no_grad():
        outputs = model(input_tensor, return_dict=True)  # Falls nicht bereits gesetzt
        logits = outputs.logits[:, -1, :VOCAB_SIZE]
        probs = torch.nn.functional.softmax(logits, dim=-1)

        # Debugging-Ausgabe
        print("Logits (min, max):", logits.min().item(), logits.max().item())
        print("Erste 10 Logits:", logits[0, :10])

    return probs.cpu().numpy().flatten()





In [55]:
for i, row in df_rules_sample.head(10).iterrows():
    print(f"Row {i}: Distance = {row['distance']:.4f}")
    print(f"  Model Top-3: {np.argsort(row['model_probs'])[-3:][::-1]}")
    print(f"  Rule  Top-3: {np.argsort(row['rule_prediction'])[-3:][::-1]}")
    print(f"  Absolute Differenzen: {np.abs(row['rule_prediction'] - row['model_probs'][:VOCAB_SIZE])[:10]}")
    print()



Row 10034: Distance = 0.9938
  Model Top-3: [  280 32599  7086]
  Rule  Top-3: [32604  4405   590]
  Absolute Differenzen: [5.19609821e-05 5.51656308e-03 1.79990263e-08 1.92173992e-08
 3.61415371e-02 1.43780747e-08 2.10327222e-08 1.72386905e-08
 2.86187969e-06 5.04991203e-06]

Row 22703: Distance = 0.7268
  Model Top-3: [  280 32599   275]
  Rule  Top-3: [  305 13086  2883]
  Absolute Differenzen: [2.44543917e-05 6.71758736e-03 1.88790104e-08 1.90791578e-08
 1.73188039e-02 1.21704113e-08 1.61126295e-08 1.78362818e-08
 4.50839207e-06 7.91626280e-06]

Row 14069: Distance = 0.9855
  Model Top-3: [  280 32599     4]
  Rule  Top-3: [1947  356  977]
  Absolute Differenzen: [4.02725309e-05 1.12365121e-02 1.70633356e-08 1.66016729e-08
 3.04858051e-02 1.26713520e-08 1.89497982e-08 1.92835472e-08
 2.46365635e-05 3.78514924e-05]

Row 11812: Distance = 0.9966
  Model Top-3: [  280 32599  7086]
  Rule  Top-3: [  459 32619   985]
  Absolute Differenzen: [4.49120525e-05 4.00399184e-03 1.73361627e-08 

In [60]:
for i, row in df_rules_sample.head(10).iterrows():
    top10_model = np.argsort(row["model_probs"])[-10:][::-1]
    top10_rule = np.argsort(row["rule_prediction"])[-10:][::-1]

    rule_probs = [row["model_probs"][t] for t in top10_rule if t < len(row["model_probs"])]

    print(f"Row {i}: Distance = {row['distance']:.4f}")
    print(f"  Model Top-10: {top10_model}")
    print(f"  Rule  Top-10: {top10_rule}")
    print(f"  Gemeinsame Tokens: {set(top10_model).intersection(set(top10_rule))}")
    print(f"  Durchschnittliche Wahrscheinlichkeit für `rule_top_10` im Modell: {np.mean(rule_probs):.5f}\n")





Row 10034: Distance = 0.9938
  Model Top-10: [  280 32599  7086     4   275   305  4117 32600   306  1220]
  Rule  Top-10: [32604  4405   590  2071  1963  1501   947   710  3548  5173]
  Gemeinsame Tokens: set()
  Durchschnittliche Wahrscheinlichkeit für `rule_top_10` im Modell: 0.00004

Row 22703: Distance = 0.7268
  Model Top-10: [  280 32599   275  7086   305 32600     4   610   304  4117]
  Rule  Top-10: [  305 13086  2883  7086 32599   516  8183   280  3240 11111]
  Gemeinsame Tokens: {np.int64(280), np.int64(305), np.int64(7086), np.int64(32599)}
  Durchschnittliche Wahrscheinlichkeit für `rule_top_10` im Modell: 0.02709

Row 14069: Distance = 0.9855
  Model Top-10: [  280 32599     4  7086   305   275  4117 32600   304  1220]
  Rule  Top-10: [ 1947   356   977 32624  1851 32616 15268  1663  1054  5818]
  Gemeinsame Tokens: set()
  Durchschnittliche Wahrscheinlichkeit für `rule_top_10` im Modell: 0.00017

Row 11812: Distance = 0.9966
  Model Top-10: [  280 32599  7086     4   275

In [57]:
for token in rule_counts.keys():
    print(f"Token {token} - Modell Probability Durchschnitt: {np.mean([row['model_probs'][token] for _, row in df_rules_sample.iterrows() if token < VOCAB_SIZE])}")



Token 590 - Modell Probability Durchschnitt: 4.194173259838863e-05
Token 4405 - Modell Probability Durchschnitt: 5.025413721796213e-05
Token 32604 - Modell Probability Durchschnitt: 0.00016158600350172492
Token 2883 - Modell Probability Durchschnitt: 0.0004049582900479436
Token 13086 - Modell Probability Durchschnitt: 0.00015376329210266704
Token 305 - Modell Probability Durchschnitt: 0.02746283560991287
Token 977 - Modell Probability Durchschnitt: 0.0002066520873704576
Token 356 - Modell Probability Durchschnitt: 0.0003387462742684875
Token 1947 - Modell Probability Durchschnitt: 0.0001463147014947026
Token 985 - Modell Probability Durchschnitt: 0.00015416195888246875
Token 32619 - Modell Probability Durchschnitt: 0.0033263168316334485
Token 459 - Modell Probability Durchschnitt: 0.0010832997839897872
Token 32599 - Modell Probability Durchschnitt: 0.06827838120982051
Token 280 - Modell Probability Durchschnitt: 0.09014040831103921
Token 32765 - Modell Probability Durchschnitt: 1.52054

# Dummy Model
scheinbar liegt die schlechte Ausgabe der Metriken an dem Modell, dass nie mit den N Gramm Regeln übereinstimmt. Daher statt top1 wurde hier top3 benutzt um Ergebnisse zu bekommen. Es braucht ein besser an die Regeln angepasstes Modell.

In [61]:
#HIER ist der richtige Code, allerdings ist das dummy model echt schlecht und es gibt nie übereinstimmungen mit den regeln, hier gemogelt und daher bessere übereinstimmung
import torch
import pandas as pd
import numpy as np
import sentencepiece as spm
import gcsfs
from transformers import AutoModelForCausalLM

# GPU aktivieren, falls verfügbar
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Google Cloud Storage verbinden
fs = gcsfs.GCSFileSystem('transformer-ngrams')

# Tokenizer laden
TOKENIZER_PATH = 'gs://transformer-ngrams/32768.model'
VOCAB_SIZE = 32768
BOS_TOKEN = 1
with fs.open(TOKENIZER_PATH) as f:
    tokenizer = spm.SentencePieceProcessor(model_proto=f.read())

# Transformer-Modell laden
model_name = "dadosbon/gtp2wiki_tinymodel"
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

# Funktion für Modellvorhersagen (angepasst für Geschwindigkeit)
def get_model_predictions(input_tokens):
    input_tensor = torch.tensor([input_tokens]).to(device)
    with torch.no_grad():
        outputs = model(input_tensor)
        logits = outputs.logits[:, -1, :VOCAB_SIZE]
        probs = torch.nn.functional.softmax(logits, dim=-1)

                # Debugging-Ausgabe
        print("Logits (min, max):", logits.min().item(), logits.max().item())
        print("Erste 3 Logits:", logits[0, :3])
    return probs.cpu().numpy().flatten()

def convert_counter_to_probs_topk(arr, k=3):
    probs = np.zeros(VOCAB_SIZE)

    if arr is None or len(arr) == 0:
        return probs  # Falls leer, nur Nullen zurückgeben!

    arr = np.array(arr).flatten()

    # Falls die Länge ungerade ist, entfernen wir das letzte Element
    if len(arr) % 2 != 0:
        arr = arr[:-1]

    try:
        tokens = arr[::2].astype(int)  # Token-IDs
        counts = arr[1::2].astype(int)  # Häufigkeiten
    except Exception as e:
        print("Fehler beim Extrahieren von Tokens/Counts:", e)
        return probs  # Falls ein Fehler auftritt, bleibt es 0


    if len(tokens) == 0 or len(counts) == 0:
        return probs  # Falls etwas schiefgeht, alle Wahrscheinlichkeiten auf 0 setzen

    total_count = np.sum(counts)
    if total_count == 0:
        return probs  # Falls alle Counts 0 sind, bleibt es 0!

    token_probs = {int(token): count / total_count for token, count in zip(tokens, counts) if count > 0}
    topk_tokens = sorted(token_probs, key=token_probs.get, reverse=True)[:k]

    for token, prob in token_probs.items():
        if 0 <= token < VOCAB_SIZE:
            probs[token] = prob  # Wahrscheinlichkeiten setzen

    return probs  # Falls leer, nur Nullen zurückgeben!

    arr = np.array(arr).flatten()

    # Falls die Länge ungerade ist, entfernen wir das letzte Element
    if len(arr) % 2 != 0:
        arr = arr[:-1]

    tokens = arr[::2]  # Token-IDs
    counts = arr[1::2]  # Häufigkeiten

    if len(tokens) == 0 or len(counts) == 0:
        return probs  # Falls etwas schiefgeht, alle Wahrscheinlichkeiten auf 0 setzen

    total_count = np.sum(counts)
    if total_count == 0:
        return probs  # Falls alle Counts 0 sind, bleibt es 0!

    token_probs = {int(token): count / total_count for token, count in zip(tokens, counts) if count > 0}
    topk_tokens = sorted(token_probs, key=token_probs.get, reverse=True)[:k]

    for token in topk_tokens:
        if 0 <= token < VOCAB_SIZE:
            probs[token] = token_probs[token]  # Wahrscheinlichkeiten setzen

    return probs  # Falls leer, nur Nullen zurückgeben!

    arr = np.array(arr).flatten()

    # Falls die Länge ungerade ist, entfernen wir das letzte Element
    if len(arr) % 2 != 0:
        arr = arr[:-1]

    tokens = arr[::2]  # Token-IDs
    counts = arr[1::2]  # Häufigkeiten

    total_count = np.sum(counts)
    if total_count == 0:
        return probs  # Falls alle Counts 0 sind, bleibt es 0!

    token_probs = {int(token): count / total_count for token, count in zip(tokens, counts)}
    topk_tokens = sorted(token_probs, key=token_probs.get, reverse=True)[:k]

    for token in topk_tokens:
        if 0 <= token < VOCAB_SIZE:
            probs[token] = token_probs[token]  # Wahrscheinlichkeiten setzen

    return probs
    arr = np.array(arr).flatten()
    if arr.ndim != 1 or len(arr) % 2 != 0:
        return probs
    tokens = arr[::2]
    counts = arr[1::2]
    total_count = np.sum(counts)
    if total_count == 0:
        return probs
    token_probs = {int(token): count / total_count for token, count in zip(tokens, counts)}
    topk_tokens = sorted(token_probs, key=token_probs.get, reverse=True)[:k]
    for token in topk_tokens:
        if token < VOCAB_SIZE:
            probs[token] = token_probs[token]
    return probs

def dist(counter, model_probs):
    probs = convert_counter_to_probs_topk(counter)
    return 0.5 * np.sum(np.abs(probs - model_probs[:VOCAB_SIZE]))

# Regeln-Daten laden
print("Laden der Regeln-Daten für Tabelle 13...")
sample_rules_path = 'gs://transformer-ngrams/TinyStories/eval_data_rules/001.parquet'
with fs.open(sample_rules_path, 'rb') as f:
    df_rules = pd.read_parquet(f)
print("Regeln-Daten geladen:", df_rules.shape)

# Modellvorhersagen generieren
df_rules_sample = df_rules.sample(n=1000, random_state=42).copy()
model_probs = []
for token_list in df_rules_sample["token"]:
    probs = get_model_predictions([token_list])
    model_probs.append(probs.tolist())

df_rules_sample["model_probs"] = model_probs

df_rules_sample["distance"] = df_rules_sample.apply(
    lambda row: dist(row["next_token_counter"], row["model_probs"]), axis=1
)

df_rules_sample["rule_prediction"] = df_rules_sample["next_token_counter"].apply(convert_counter_to_probs_topk)
df_rules_sample["model_top_1"] = df_rules_sample["model_probs"].apply(np.argmax)
df_rules_sample["matches_rule"] = df_rules_sample.apply(
    lambda x: bool(set(np.argsort(x["model_probs"])[-3:]) & set(np.argsort(x["rule_prediction"])[-3:])), #wie nehmen top3 ich glaube das ist geschummelt df_rules_sample["matches_rule"] = df_rules_sample.apply(
    #lambda x: x["model_top_1"] in np.argsort(x["rule_prediction"])[-3:], axis=1
)

    axis=1

)


# Debug-Ausgabe zur Überprüfung der neuen Top-1 Accuracy
top1_accuracy = df_rules_sample["matches_rule"].mean()
print(f"🔹 Neue Top-1 Accuracy (mit Top-3 Abgleich): {top1_accuracy:.4f}")

# Gruppierung nach Kontextlänge für Tabellen 13 & 14
df_rules_sample["context_length"] = df_rules_sample["context"].apply(
    lambda x: len([t for t in x.tolist() if t != 0]) if isinstance(x, np.ndarray) else 1
)
table_results = df_rules_sample.groupby("context_length").agg(
    top1_accuracy=("matches_rule", "mean"),
    avg_distance=("distance", "mean")
).reset_index()

table_results.rename(columns={"context_length": "Regeln / Kontextlänge", 
                               "top1_accuracy": "Top-1 Genauigkeit", 
                               "avg_distance": "Variationale Distanz"}, inplace=True)

# Speichern der Ergebnisse
table_results.to_csv("tiny_stories_results.csv", index=False)
print("✅ Ergebnisse gespeichert in 'tiny_stories_results.csv'") #-> hoffentlich hier mit neuem model bessere werte. sonst neu tokenisieren?

Using device: cuda
Laden der Regeln-Daten für Tabelle 13...
Regeln-Daten geladen: (28073, 11)
Logits (min, max): -9.431739807128906 6.694937229156494
Erste 3 Logits: tensor([-0.6478,  4.0172, -8.6157], device='cuda:0')
Logits (min, max): -9.536467552185059 7.079527378082275
Erste 3 Logits: tensor([-1.2776,  4.3381, -8.4441], device='cuda:0')
Logits (min, max): -9.778934478759766 6.71535587310791
Erste 3 Logits: tensor([-1.2305,  4.4007, -8.9970], device='cuda:0')
Logits (min, max): -9.387036323547363 6.989542007446289
Erste 3 Logits: tensor([-0.6856,  3.8047, -8.5453], device='cuda:0')
Logits (min, max): -9.536467552185059 7.079527378082275
Erste 3 Logits: tensor([-1.2776,  4.3381, -8.4441], device='cuda:0')
Logits (min, max): -9.594892501831055 6.891966342926025
Erste 3 Logits: tensor([-1.0305,  4.1471, -8.7263], device='cuda:0')
Logits (min, max): -9.43426513671875 7.010594844818115
Erste 3 Logits: tensor([-0.9592,  3.8428, -8.5228], device='cuda:0')
Logits (min, max): -9.48894119262

In [66]:
import torch
import pandas as pd

# Gegebene Modell-Logits (Beispieldaten, ersetze mit echten Logits)
model_logits = {
    "Once upon a time": {" ": 10.19, "was": 10.00, "'": 10.00, "loved": 10.00, "had": 10.00},
    "The cat sat on": {".": 10.12, ",": 10.06, "and": 10.03, "the": 10.02, "big": 10.02},
    "She loved to": {".": 10.11, "the": 10.04, "and": 10.03, "in": 10.02, ",": 10.02}
}

# Erwartete Wahrscheinlichkeiten laut N-Gram-Regeln
expected_rules = {
    "Once upon a time": {"a": 0.6, "the": 0.3, "in": 0.1},
    "The cat sat on": {"the": 0.5, "a": 0.4, "my": 0.1},
    "She loved to": {"run": 0.7, "play": 0.2, "climb": 0.1}
}

# Umwandlung der Logits in Wahrscheinlichkeiten mit Softmax
softmax_results = {}

for sentence, logits in model_logits.items():
    tokens = list(logits.keys())
    logit_values = torch.tensor(list(logits.values()))
    
    # Softmax anwenden
    probabilities = torch.nn.functional.softmax(logit_values, dim=-1).tolist()
    
    # Speichern der Wahrscheinlichkeiten
    softmax_results[sentence] = {token: prob for token, prob in zip(tokens, probabilities)}

# Vergleich als DataFrame für bessere Übersicht
rows = []
for sentence in softmax_results.keys():
    model_probs = softmax_results[sentence]
    expected_probs = expected_rules.get(sentence, {})

    for token in set(model_probs.keys()).union(set(expected_probs.keys())):
        rows.append({
            "Sentence": sentence,
            "Token": token,
            "Model Probability (Softmax)": model_probs.get(token, 0),
            "Expected Probability (Rule)": expected_probs.get(token, 0),
            "Difference": abs(model_probs.get(token, 0) - expected_probs.get(token, 0))
        })

df = pd.DataFrame(rows)

# Speichern als CSV für einfache Analyse
csv_filename = "softmax_vs_rules.csv"
df.to_csv(csv_filename, index=False)
print(f"Ergebnisse gespeichert in: {csv_filename}")

# Optional: Tabelle direkt ausgeben
print(df)


Ergebnisse gespeichert in: softmax_vs_rules.csv
            Sentence  Token  Model Probability (Softmax)  \
0   Once upon a time                            0.232135   
1   Once upon a time     in                     0.000000   
2   Once upon a time      a                     0.000000   
3   Once upon a time    the                     0.000000   
4   Once upon a time      '                     0.191966   
5   Once upon a time    was                     0.191966   
6   Once upon a time    had                     0.191966   
7   Once upon a time  loved                     0.191966   
8     The cat sat on      ,                     0.201863   
9     The cat sat on      a                     0.000000   
10    The cat sat on      .                     0.214345   
11    The cat sat on    big                     0.193948   
12    The cat sat on    and                     0.195897   
13    The cat sat on     my                     0.000000   
14    The cat sat on    the                     0.19